# TimeEval shared parameter optimization result analysis

In [1]:
# Automatically reload packages:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import json
import warnings
import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as py
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from pathlib import Path
from timeeval import Datasets

## Configuration

Target parameters that were optimized in this run (per algorithm):

In [3]:
algo_param_mapping = {
  "FFT": ["context_window_size"],
  "Subsequence LOF": ["n_neighbors", "leaf_size"],
  "Spectral Residual (SR)": ["mag_window_size", "score_window_size"],
  "LaserDBN": ["n_bins"],
  "k-Means": ["n_clusters"],
  "XGBoosting (RR)": ["n_estimators", "train_window_size", "n_trees"],
  "Hybrid KNN": ["n_neighbors", "n_estimators"],
  "Subsequence IF": ["n_trees"],
  "DeepAnT": ["prediction_window_size"],
  "Random Forest Regressor (RR)": ["train_window_size", "n_trees"]
}

Define data and results folder:

In [4]:
# constants and configuration
data_path = Path("../../data") / "test-cases"
result_root_path = Path("../timeeval_experiments/results")
experiment_result_folder = "2021-09-27_shared-optim"

# build paths
result_paths = [d for d in result_root_path.iterdir() if d.is_dir()]
print("Available result directories:")
display(result_paths)

result_path = result_root_path / experiment_result_folder
print("\nSelecting:")
print(f"Data path: {data_path.resolve()}")
print(f"Result path: {result_path.resolve()}")

Available result directories:


[PosixPath('../timeeval_experiments/results/2021-09-30-torsk'),
 PosixPath('../timeeval_experiments/results/2021-09-27_shared-optim')]


Selecting:
Data path: /home/sebastian/Documents/Projects/akita/data/test-cases
Result path: /home/sebastian/Documents/Projects/akita/timeeval/timeeval_experiments/results/2021-09-27_shared-optim


Load results and dataset metadata:

In [5]:
def extract_hyper_params(param_names):
    def extract(value):
        params = json.loads(value)
        result = ""
        for name in param_names:
            value = params[name]
            result += f"{name}={value},"
        return "".join(result.rsplit(",", 1))
    return extract

# load results
print(f"Reading results from {result_path.resolve()}")
df = pd.read_csv(result_path / "results.csv")

# add dataset_name column
df["dataset_name"] = df["dataset"].str.split(".").str[0]

# add optim_params column
df["optim_params"] = ""
for algo in algo_param_mapping:
    df_algo = df.loc[df["algorithm"] == algo]
    df.loc[df_algo.index, "optim_params"] = df_algo["hyper_params"].apply(extract_hyper_params(algo_param_mapping[algo]))

# load dataset metadata
dmgr = Datasets(data_path)

Reading results from /home/sebastian/Documents/Projects/akita/timeeval/timeeval_experiments/results/2021-09-27_shared-optim


Define plotting functions:

In [6]:
def load_scores_df(algorithm_name, dataset_id, optim_params, repetition=1):
    params_id = df.loc[(df["algorithm"] == algorithm_name) & (df["collection"] == dataset_id[0]) & (df["dataset"] == dataset_id[1]) & (df["optim_params"] == optim_params), "hyper_params_id"].item()
    path = (
        result_path /
        algorithm_name /
        params_id /
        dataset_id[0] /
        dataset_id[1] /
        str(repetition) /
        "anomaly_scores.ts"
    )
    return pd.read_csv(path, header=None)

def plot_scores(algorithm_name, dataset_name):
    if isinstance(algorithm_name, tuple):
        algorithms = [algorithm_name]
    elif not isinstance(algorithm_name, list):
        raise ValueError("Please supply a tuple (algorithm_name, optim_params) or a list thereof as first argument!")
    else:
        algorithms = algorithm_name
    # construct dataset ID
    dataset_id = ("GutenTAG", f"{dataset_name}.unsupervised")

    # load dataset details
    df_dataset = dmgr.get_dataset_df(dataset_id)

    # check if dataset is multivariate
    dataset_dim = df.loc[df["dataset_name"] == dataset_name, "dataset_input_dimensionality"].unique().item()
    dataset_dim = dataset_dim.lower()
    
    auroc = {}
    df_scores = pd.DataFrame(index=df_dataset.index)
    skip_algos = []
    for algo, optim_params in algorithms:
        # get algorithm metric results
        try:
            auroc[(algo, optim_params)] = df.loc[
                (df["algorithm"] == algo) & (df["dataset_name"] == dataset_name) & (df["optim_params"] == optim_params),
                "ROC_AUC"
            ].item()
        except ValueError:
            warnings.warn(f"No ROC_AUC score found! Probably {algo} with params {optim_params} was not executed on {dataset_name}.")
            auroc[(algo, optim_params)] = -1
            skip_algos.append((algo, optim_params))
            continue

        # load scores
        training_type = df.loc[df["algorithm"] == algo, "algo_training_type"].values[0].lower().replace("_", "-")
        try:
            df_scores[(algo, optim_params)] = load_scores_df(algo, ("GutenTAG", f"{dataset_name}.{training_type}"), optim_params).iloc[:, 0]
        except (ValueError, FileNotFoundError):
            warnings.warn(f"No anomaly scores found! Probably {algo} was not executed on {dataset_name} with params {optim_params}.")
            df_scores[(algo, optim_params)] = np.nan
            skip_algos.append((algo, optim_params))
    algorithms = [a for a in algorithms if a not in skip_algos]

    # Create plot
    fig = make_subplots(2, 1)
    if dataset_dim == "multivariate":
        for i in range(1, df_dataset.shape[1]-1):
            fig.add_trace(go.Scatter(x=df_dataset.index, y=df_dataset.iloc[:, i], name=f"channel-{i}"), 1, 1)
    else:
        fig.add_trace(go.Scatter(x=df_dataset.index, y=df_dataset.iloc[:, 1], name="timeseries"), 1, 1)
    fig.add_trace(go.Scatter(x=df_dataset.index, y=df_dataset["is_anomaly"], name="label"), 2, 1)
    
    for item in algorithms:
        algo, optim_params = item
        fig.add_trace(go.Scatter(x=df_scores.index, y=df_scores[item], name=f"{algo}={auroc[item]:.4f} ({optim_params})"), 2, 1)
    fig.update_xaxes(matches="x")
    fig.update_layout(
        title=f"Results of {','.join(np.unique([a for a, _ in algorithms]))} on {dataset_name}",
        height=400
    )
    return py.iplot(fig)

## Analyze TimeEval results

In [7]:
df[["algorithm", "dataset_name", "status", "AVERAGE_PRECISION", "PR_AUC", "RANGE_PR_AUC", "ROC_AUC", "execute_main_time", "optim_params"]]

,algorithm,dataset_name,status,AVERAGE_PRECISION,PR_AUC,RANGE_PR_AUC,ROC_AUC,execute_main_time,optim_params
0,DeepAnT,ecg-channels-all-of-3,Status.OK,0.005201,0.005145,0.485061,0.082442,10.334360,prediction_window_size=1
1,DeepAnT,ecg-channels-single-of-10,Status.OK,0.006335,0.006263,0.237304,0.256400,10.481046,prediction_window_size=1
2,DeepAnT,ecg-channels-single-of-2,Status.OK,0.005987,0.005920,0.213420,0.213586,10.229906,prediction_window_size=1
3,DeepAnT,ecg-channels-single-of-20,Status.OK,0.006699,0.006619,0.247859,0.303753,10.746507,prediction_window_size=1
4,DeepAnT,ecg-channels-single-of-5,Status.OK,0.006205,0.006135,0.276975,0.237219,10.280138,prediction_window_size=1
...,...,...,...,...,...,...,...,...,...
29860,Subsequence LOF,sinus-type-pattern-shift,Status.OK,0.659779,0.657385,0.637168,0.986935,11.559971,"n_neighbors=50,leaf_size=40"
29861,Subsequence LOF,sinus-type-pattern,Status.OK,1.000000,1.000000,0.995035,1.000000,11.696036,"n_neighbors=50,leaf_size=40"
29862,Subsequence LOF,sinus-type-platform,Status.OK,0.347330,0.339532,0.337769,0.816412,13.332605,"n_neighbors=50,leaf_size=40"
29863,Subsequence LOF,sinus-type-trend,Status.OK,0.996865,0.996852,0.985046,0.999963,15.411443,"n_neighbors=50,leaf_size=40"


---

### Errors

In [8]:
df_error_counts = df.pivot_table(index=["algo_training_type", "algorithm"], columns=["status"], values="repetition", aggfunc="count")
df_error_counts = df_error_counts.fillna(value=0).astype(np.int64)

#### Aggregation of errors per algorithm grouped by algorithm training type

In [9]:
for tpe in ["SEMI_SUPERVISED", "SUPERVISED", "UNSUPERVISED"]:
    if tpe in df_error_counts.index:
        print(tpe)
        display(df_error_counts.loc[tpe])

SEMI_SUPERVISED


status,Status.ERROR,Status.OK,Status.TIMEOUT
algorithm,,,
DeepAnT,134,2346,0
Hybrid KNN,765,2025,0
LaserDBN,35,422,8
Random Forest Regressor (RR),0,1620,0
XGBoosting (RR),0,2552,2308


UNSUPERVISED


status,Status.ERROR,Status.OK,Status.TIMEOUT
algorithm,,,
FFT,0,675,0
Spectral Residual (SR),0,4860,0
Subsequence IF,0,1620,0
Subsequence LOF,0,9720,0
k-Means,0,775,0


#### Slow algorithms

Algorithms, for which more than 50% of all executions ran into the timeout.

In [10]:
df_error_counts[df_error_counts["Status.TIMEOUT"] > (df_error_counts["Status.ERROR"] + df_error_counts["Status.OK"])]

,status,Status.ERROR,Status.OK,Status.TIMEOUT
algo_training_type,algorithm,,,


#### Broken algorithms

Algorithms, which failed for at least 50% of the executions.

In [11]:
error_threshold = 0.5
df_error_counts[df_error_counts["Status.ERROR"] > error_threshold*(
    df_error_counts["Status.TIMEOUT"] + df_error_counts["Status.ERROR"] + df_error_counts["Status.OK"]
)]

,status,Status.ERROR,Status.OK,Status.TIMEOUT
algo_training_type,algorithm,,,


#### Detail errors

In [12]:
algo_list = ["DeepAnT", "Hybrid KNN", "LaserDBN"]

error_list = ["OOM", "Segfault", "ZeroDivisionError", "IncompatibleParameterConfig", "WrongDBNState", "other"]
errors = pd.DataFrame(0, index=error_list, columns=algo_list, dtype=np.int_)
for algo in algo_list:
    df_tmp = df[(df["algorithm"] == algo) & (df["status"] == "Status.ERROR")]
    for i, run in df_tmp.iterrows():
        path = result_path / run["algorithm"] / run["hyper_params_id"] / run["collection"] / run["dataset"] / str(run["repetition"]) / "execution.log"
        with path.open("r") as fh:
            log = fh.read()
            if "status code '139'" in log:
                errors.loc["Segfault", algo] += 1
            elif "status code '137'" in log:
                errors.loc["OOM", algo] += 1
            elif "Expected n_neighbors <= n_samples" in log:
                errors.loc["IncompatibleParameterConfig", algo] += 1
            elif "ZeroDivisionError" in log:
                errors.loc["ZeroDivisionError", algo] += 1
            elif "does not have key" in log:
                errors.loc["WrongDBNState", algo] += 1
            else:
                print(f'\n\n#### {run["dataset"]} ({run["optim_params"]})')
                print(log)
                errors.loc["other", algo] += 1
errors.T

,OOM,Segfault,ZeroDivisionError,IncompatibleParameterConfig,WrongDBNState,other
DeepAnT,94,0,40,0,0,0
Hybrid KNN,0,0,0,765,0,0
LaserDBN,0,20,0,0,15,0


---

### Parameter assessment

In [13]:
sort_by = ("ROC_AUC", "mean")
metric_agg_type = ["mean", "median"]
time_agg_type = "mean"
aggs = {
    "AVERAGE_PRECISION": metric_agg_type,
    "RANGE_PR_AUC": metric_agg_type,
    "PR_AUC": metric_agg_type,
    "ROC_AUC": metric_agg_type,
    "train_main_time": time_agg_type,
    "execute_main_time": time_agg_type,
    "repetition": "count"
}

df_tmp = df.reset_index()
df_tmp = df_tmp.groupby(by=["algorithm", "optim_params"]).agg(aggs)
df_tmp = (df_tmp
          .reset_index()
          .sort_values(by=["algorithm", sort_by], ascending=False)
          .set_index(["algorithm", "optim_params"]))
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_tmp)

AVERAGE_PRECISION  \
                                                                                             mean   
algorithm                    optim_params                                                           
k-Means                      n_clusters=50                                               0.731988   
                             n_clusters=10                                               0.633051   
                             n_clusters=40                                               0.735390   
                             n_clusters=30                                               0.717902   
                             n_clusters=5                                                0.508202   
XGBoosting (RR)              n_estimators=100,train_window_size=500,n_trees=10           0.577270   
                             n_estimators=1000,train_window_size=500,n_trees=10          0.595681   
                             n_estimators=100,train_window_size=1000,n_trees=10          0.575332   
                             n_estimators=1000,train_window_size=100,n_trees=10          0.543180   
                             n_estimators=100,train_window_size=100,n_trees=10           0.539758   
                             n_estimators=100,train_window_size=50,n_trees=1000          0.580512   
                             n_estimators=100,train_window_size=100,n_trees=100          0.539697   
                             n_estimators=1000,train_window_size=50,n_trees=100          0.577064   
                             n_estimators=100,train_window_size=500,n_trees=100          0.566974   
                             n_estimators=1000,train_window_size=50,n_trees=10           0.537368   
                             n_estimators=100,train_window_size=50,n_trees=100           0.529155   
                             n_estimators=100,train_window_size=50,n_trees=10            0.529162   
                             n_estimators=10,train_window_size=500,n_trees=1000          0.229250   
                             n_estimators=10,train_window_size=1000,n_trees=100          0.179102   
                             n_estimators=10,train_window_size=1000,n_trees=10           0.179040   
                             n_estimators=10,train_window_size=500,n_trees=100           0.174391   
                             n_estimators=10,train_window_size=500,n_trees=10            0.174494   
                             n_estimators=10,train_window_size=100,n_trees=1000          0.174394   
                             n_estimators=10,train_window_size=100,n_trees=100           0.174482   
                             n_estimators=10,train_window_size=100,n_trees=10            0.174541   
                             n_estimators=10,train_window_size=50,n_trees=10             0.170264   
                             n_estimators=10,train_window_size=50,n_trees=1000           0.170302   
                             n_estimators=10,train_window_size=50,n_trees=100            0.170352   
                             n_estimators=10,train_window_size=1000,n_trees=...               NaN   
                             n_estimators=100,train_window_size=100,n_trees=...               NaN   
                             n_estimators=100,train_window_size=1000,n_trees...               NaN   
                             n_estimators=100,train_window_size=1000,n_trees...               NaN   
                             n_estimators=100,train_window_size=500,n_trees=...               NaN   
                             n_estimators=1000,train_window_size=100,n_trees...               NaN   
                             n_estimators=1000,train_window_size=100,n_trees...               NaN   
                             n_estimators=1000,train_window_size=1000,n_tree...               NaN   
                             n_estimators=1000,train_window_size=1000,n_tree...               NaN   
                             n_estimator

#### Selected parameters

- k-Means: `n_clusters=50` (more are usually better)
- XGBoosting (RR): `n_estimators=500,train_window_size=500,n_trees=10` (more estimators are better)
- Subsequence LOF: `n_neighbors=50,leaf_size=20` (robust to leaf_size)
- Subsequence IF: `n_trees=100`
- Spectral Residual (SR): `mag_window_size=40,score_window_size=40` (robust, but bad performance)
- Random Forest Regressor (RR): `train_window_size=500,n_trees=500` (more trees are better)
- LaserDBN: `n_bins=10` (more are better; marginal improvement)
- Hybrid KNN: `n_neighbors=10,n_estimators=1000` (less neighbors and more estimators are better)
- FFT: `context_window_size=5` (robust, but bad performance)
- DeepAnT: `prediction_window_size=50`

Summary:

- n_clusters=50
- n_estimators=500
- train_window_size=500
- n_trees=500
- n_neighbors=50
- mag_window_size=40
- score_window_size=40
- prediction_window_size=50
- n_bins=10 (**re-test for other algorithms!**)
- context_window_size=5 (**re-test for other algorithms!**)
- Overwrites for Hybrid KNN: `n_neighbors=10,n_estimators=1000`
- Overwrites for XGBoosting (RR): `n_trees=10`

In [ ]:
plot_scores([("k-Means", "n_clusters=50"), ("k-Means", "n_clusters=5")], "ecg-channels-single-of-5")

---

### Window size parameter assessment

In [15]:
algo_list = ["Subsequence LOF", "Subsequence IF", "Spectral Residual (SR)", "DeepAnT"]
df2 = df[df["algorithm"].isin(algo_list)].copy()

# overwrite optim_params column
df2 = df2.drop(columns=["optim_params"])
df2["window_size"] = ""
for algo in algo_list:
    df_algo = df2.loc[df2["algorithm"] == algo]
    df2.loc[df_algo.index, "window_size"] = df_algo["hyper_params"].apply(extract_hyper_params(["window_size"]))
df2["window_size"] = df2["window_size"].str.split("=").apply(lambda v: v[1]).astype(int)
df2["period_size"] = df2["dataset"].apply(lambda d: dmgr.get(("GutenTAG", d)).period_size)
df2["window_size_group"] = df2["window_size"] / df2["period_size"]
df2["window_size_group"] = (df2["window_size_group"]
                            .fillna(df2["window_size"])
                            .round(1)
                            .replace(50., 0.5)
                            .replace(100, 1.0)
                            .replace(150, 1.5)
                            .replace(200, 2.0))
df2 = df2.drop(columns=["window_size", "period_size"])
df2

,algorithm,collection,dataset,algo_training_type,algo_input_dimensionality,dataset_training_type,dataset_input_dimensionality,train_preprocess_time,train_main_time,execute_preprocess_time,...,error_message,repetition,hyper_params,hyper_params_id,ROC_AUC,PR_AUC,RANGE_PR_AUC,AVERAGE_PRECISION,dataset_name,window_size_group
0,DeepAnT,GutenTAG,ecg-channels-all-of-3.semi-supervised,SEMI_SUPERVISED,MULTIVARIATE,SEMI_SUPERVISED,MULTIVARIATE,NaN,3490.080107,NaN,...,NaN,1,"{""batch_size"": 64, ""early_stopping_delta"": 0.0...",c19923440357dc06942f10287ac3b6e4,0.082442,0.005145,0.485061,0.005201,ecg-channels-all-of-3,0.5
1,DeepAnT,GutenTAG,ecg-channels-single-of-10.semi-supervised,SEMI_SUPERVISED,MULTIVARIATE,SEMI_SUPERVISED,MULTIVARIATE,NaN,3136.354387,NaN,...,NaN,1,"{""batch_size"": 64, ""early_stopping_delta"": 0.0...",c19923440357dc06942f10287ac3b6e4,0.256400,0.006263,0.237304,0.006335,ecg-channels-single-of-10,0.5
2,DeepAnT,GutenTAG,ecg-channels-single-of-2.semi-supervised,SEMI_SUPERVISED,MULTIVARIATE,SEMI_SUPERVISED,MULTIVARIATE,NaN,2725.753779,NaN,...,NaN,1,"{""batch_size"": 64, ""early_stopping_delta"": 0.0...",c19923440357dc06942f10287ac3b6e4,0.213586,0.005920,0.213420,0.005987,ecg-channels-single-of-2,0.5
3,DeepAnT,GutenTAG,ecg-channels-single-of-20.semi-supervised,SEMI_SUPERVISED,MULTIVARIATE,SEMI_SUPERVISED,MULTIVARIATE,NaN,2572.371926,NaN,...,NaN,1,"{""batch_size"": 64, ""early_stopping_delta"": 0.0...",c19923440357dc06942f10287ac3b6e4,0.303753,0.006619,0.247859,0.006699,ecg-channels-single-of-20,0.5
4,DeepAnT,GutenTAG,ecg-channels-single-of-5.semi-supervised,SEMI_SUPERVISED,MULTIVARIATE,SEMI_SUPERVISED,MULTIVARIATE,NaN,1938.601979,NaN,...,NaN,1,"{""batch_size"": 64, ""early_stopping_delta"": 0.0...",c19923440357dc06942f10287ac3b6e4,0.237219,0.006135,0.276975,0.006205,ecg-channels-single-of-5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29860,Subsequence LOF,GutenTAG,sinus-type-pattern-shift.unsupervised,UNSUPERVISED,UNIVARIATE,UNSUPERVISED,UNIVARIATE,NaN,NaN,NaN,...,NaN,1,"{""distance_metric_order"": 2, ""leaf_size"": 40, ...",cdecfb673e7e0913e8a5d3910751c387,0.986935,0.657385,0.637168,0.659779,sinus-type-pattern-shift,2.0
29861,Subsequence LOF,GutenTAG,sinus-type-pattern.unsupervised,UNSUPERVISED,UNIVARIATE,UNSUPERVISED,UNIVARIATE,NaN,NaN,NaN,...,NaN,1,"{""distance_metric_order"": 2, ""leaf_size"": 40, ...",cdecfb673e7e0913e8a5d3910751c387,1.000000,1.000000,0.995035,1.000000,sinus-type-pattern,2.0
29862,Subsequence LOF,GutenTAG,sinus-type-platform.unsupervised,UNSUPERVISED,UNIVARIATE,UNSUPERVISED,UNIVARIATE,NaN,NaN,NaN,...,NaN,1,"{""distance_metric_order"": 2, ""leaf_size"": 40, ...",cdecfb673e7e0913e8a5d3910751c387,0.816412,0.339532,0.337769,0.347330,sinus-type-platform,2.0
29863,Subsequence LOF,GutenTAG,sinus-type-trend.unsupervised,UNSUPERVISED,UNIVARIATE,UNSUPERVISED,UNIVARIATE,NaN,NaN,NaN,...,NaN,1,"{""distance_metric_order"": 2, ""leaf_size"": 40, ...",cdecfb673e7e0913e8a5d3910751c387,0.999963,0.996852,0.985046,0.996865,sinus-type-trend,2.0


In [16]:
sort_by = ("ROC_AUC", "mean")
metric_agg_type = ["mean", "median"]
time_agg_type = "mean"
aggs = {
    "AVERAGE_PRECISION": metric_agg_type,
    "RANGE_PR_AUC": metric_agg_type,
    "PR_AUC": metric_agg_type,
    "ROC_AUC": metric_agg_type,
    "train_main_time": time_agg_type,
    "execute_main_time": time_agg_type,
    "index": lambda index: "" if len(index) < 2 else f"{index.iloc[0]}-{index.iloc[-1]}",
    "repetition": "count"
}

df_tmp = df2.reset_index()
df_tmp = df_tmp.groupby(by=["algorithm", "window_size_group"]).agg(aggs)
df_tmp = df_tmp.rename(columns={"index": "experiment IDs", "<lambda>": ""})
df_tmp = (df_tmp
          .reset_index()
          .sort_values(by=["algorithm", sort_by], ascending=False)
          .set_index(["algorithm", "window_size_group"]))
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_tmp)

AVERAGE_PRECISION            \
                                                      mean    median   
algorithm              window_size_group                               
Subsequence LOF        2.0                        0.677201  0.793231   
                       1.5                        0.678294  0.787024   
                       1.0                        0.710013  0.853176   
                       0.5                        0.715130  0.851845   
Subsequence IF         1.5                        0.375868  0.195873   
                       2.0                        0.385378  0.173336   
                       1.0                        0.379023  0.198106   
                       0.5                        0.301753  0.134934   
Spectral Residual (SR) 2.0                        0.133300  0.057824   
                       0.5                        0.137767  0.061300   
                       1.5                        0.136132  0.054127   
                       1.0                        0.135212  0.066699   
DeepAnT                2.0                        0.523242  0.582397   
                       1.0                        0.572737  0.612512   
                       1.5                        0.513430  0.481881   
                       0.5                        0.613091  0.857171   

                                         RANGE_PR_AUC              PR_AUC  \
                                                 mean    median      mean   
algorithm              window_size_group                                    
Subsequence LOF        2.0                   0.644477  0.665071  0.676099   
                       1.5                   0.644560  0.695121  0.677149   
                       1.0                   0.678452  0.770666  0.708901   
                       0.5                   0.671394  0.735250  0.714049   
Subsequence IF         1.5                   0.406711  0.356624  0.372936   
                       2.0                   0.434137  0.357213  0.384005   
                       1.0                   0.419387  0.358280  0.376608   
                       0.5                   0.336601  0.237578  0.298906   
Spectral Residual (SR) 2.0                   0.162506  0.064206  0.136070   
                       0.5                   0.146608  0.068046  0.136600   
                       1.5                   0.156055  0.063705  0.134636   
                       1.0                   0.145925  0.068548  0.134208   
DeepAnT                2.0                   0.520148  0.516497  0.517037   
                       1.0                   0.581578  0.584584  0.565698   
                       1.5                   0.525467  0.466834  0.507227   
                       0.5                   0.634828  0.754389  0.607782   

                                                     ROC_AUC            \
                                            median      mean    median   
algorithm              window_size_group                                 
Subsequence LOF        2.0                0.788295  0.924573  0.995574   
                       1.5                0.785187  0.918495  0.996107   
                       1.0                0.852707  0.915659  0.997009   
                       0.5                0.851434  0.909146  0.992777   
Subsequence IF         1.5                0.191174  0.794625  0.862731   
                       2.0                0.170053  0.782315  0.853456   
                       1.0                0.194197  0.781650  0.864849   
                       0.5                0.125000  0.742701  0.813636   
Spectral Residual (SR) 2.0                0.058108  0.573339  0.542867   
                       0.5                0.059844  0.569153  0.533754   
                       1.5                0.053187  0.565776  0.538426   
                       1.0                0.065900  0.540099  0.528240   
DeepAnT                2.0                0.581362  0.831119  0.987921   
                       1

#### Selected parameters

Use the heuristic `2.0 dataset period size`. It works best for SubLOF, SR, and DeepAnT. SubIF seems to perform better with 1.5 period size, but just slightly, so 2.0 should be fine.